# A CNN-LSTM framework for the solar wind density forecasting
In this notebook we pre-process solar wind and solar binary mask for the time series analysis


## Notebook Contributors
* Andrea Giuseppe Di Francesco -- email: difrancesco.1836928@studenti.uniroma1.it
* Massimo Coppotelli -- email: coppotelli.1705325@studenti.uniroma1.it

In [1]:
# !pip install pandas
# !pip install numpy
# !pip install torch

In [2]:
import pandas as pd
import numpy as np
#import torch


In [21]:

def open_dataframe(total_path):
    with open(total_path, 'r') as f:
        skippies = 0
        for line in f.readlines():
            # print(line)
            if line.startswith("#"):
                skippies += 1
            else:
                break

    return skippies  

path_elc = './wind_dataset/wind-2012/WI_ELM2_3DP_45129.csv'
path_ptr = './wind_dataset/wind-2012/OMNI_HRO_5MIN_45129.csv'
data_elc = pd.read_csv(path_elc, skiprows= open_dataframe(path_elc))

# data_elc = 
data_ptr = pd.read_csv(path_ptr, skiprows= open_dataframe(path_ptr))


/tmp/ipykernel_169747/2028443142.py:15: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  data_elc = pd.read_csv(path_elc, skiprows= open_dataframe(path_elc))
/tmp/ipykernel_169747/2028443142.py:18: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  data_ptr = pd.read_csv(path_ptr, skiprows= open_dataframe(path_ptr))


In [68]:
electron_density = data_elc['ELECTRON_NO_DENSITY_#/cm3'][:30].astype(float)
elc_sws = data_elc['MOST_PROBABLE_SPEED_ELECTRONS_km/sec'][:-3].astype(float)
# proton_density = data_ptr['PROTON_DENSITY_n/cc'][:-3].to_numpy().astype(float)
# sws = data_ptr['FLOW_SPEED__GSE_km/s'][:-3].to_numpy().astype(float)
electron_density

0     6.748405
1     7.009005
2     6.309707
3     6.925176
4     6.120157
5     6.527220
6     6.032610
7     5.995866
8     5.750294
9     5.603943
10    5.492118
11    5.990958
12    5.347048
13    5.006005
14    5.475274
15    5.541898
16    5.692333
17    5.472359
18    5.338684
19    4.681743
20    5.138120
21    5.170004
22    4.749243
23    5.216201
24    6.202234
25    7.031865
26    6.813206
27    6.186680
28    5.537101
29    5.728745
Name: ELECTRON_NO_DENSITY_#/cm3, dtype: float64

In [67]:
import plotly.express as px

fig = px.line(data_elc, x="ELECTRON_NO_DENSITY_#/cm3", y='MOST_PROBABLE_SPEED_ELECTRONS_km/sec')
fig.show()#, color='country')

[0.1124569 0.1124569 0.1124569 ... 0.1124569 0.1124569 0.1124569]
